In [1]:
import operator

### /Semantic DB (Semantic Layer Seed)

#### /Topic DB (Knowledge Base)

In [9]:
"""
목적 : 에이전트가 공유하는 지식체계

TOPIC_DB는 "KNOWLEDGE_001"과 같은 ID를 지니는 row의 "JSONB" 컬럼에 저장됨.
나머지 컬럼은 "KNOWLEDGE_ID", "VERSION", "CREATED_AT", "UPDATED_AT", "CREATOR", "SOURCE","TAG" 등 메타데이터로 구성됨.
내규, 업무분장, 주요 정관, 기준서, 매뉴얼의 경우 필수적으로 지식 DB화하여 에이전트가 참조하도록 해야함.

"""

# 에이전트 간 공유되는 지식 프레임워크 (Shared Knowledge Framework)
TOPIC_DB = {
    "metadata": {
        "domain": "Financial Risk Management",
        "last_updated": "2026-01-28",
        "authoritative_source": "IFRS 9 / Internal Credit Model"
    },
    # 에이전트가 사고의 기준으로 삼는 도메인 지식 (Fixed Knowledge)
    "taxonomy": {
        "AI_sector": {
            "description": "Artificial Intelligence and Semiconductor market",
            "risk_weight": 1.5,
            "critical_threshold": 0.85
        },
        "Market_Volatility": {
            "description": "General market fluctuation index",
            "risk_weight": 1.2,
            "critical_threshold": 0.75
        }
    },
    # 실행 중 생성되는 동적 지식 (Global Blackboard / Shared Memory)
    "dynamic_context": {
        "current_market_sentiment": "Bullish",
        "active_alerts": [],
        "aggregated_risk_score": 0.0
    }
}

#### /Task DB (Operation 제어 Layer)

In [4]:
"""
목적 : 외부화된 에이전트 프로세스 제어부 

얘는 process db 내의 한 row에서 'JSONB' 컬럼에 해당하는 내용을 정의한 것임. flatten 없이 json이 통째로 저장된 형태.
나머지 컬럼은 process id, 버전, 메타데이터, 프로세스 비즈니스 맥락, 프로세스 쿼리, 실행시간 등을 위한 태그 등등의 정보가 들어감.

실행 시에는 DB에서 이 JSON 데이터를 로드하여 사용하여 에이전트프로세스가 구동됨.
향후 JSON 내에서 다른 PROCESS를 참조하는 기능도 구현할 경우 NESTED PROCESS 실행 구조가 가능함.
따라서, 초기의 PROCESS 정의는 가장 기초적인 프로세스부터 시작하여 점진적으로 복잡한 구조로 확장해야함.
그래서 컨설턴트가 필요한 것이셈; 
"""

TASK_SPECIFICATION_DB = {
    # 1. Places (상태 저장소)
    # snakes 코드의 net.add_place()에 해당
    "places": [
        "incoming_news",  # 초기 입력 대기 (Surface)
        "consulted",      # 컨설턴트 분석 완료 (Observation)
        "evaluated",      # 슈퍼바이저 승인 완료 (Computation)
        "action_taken"    # 워커 실행 완료 (최종) (Surface)
    ],

    # 2. Roles (수행 주체)
    # 각 Transition을 담당할 에이전트 정의
    "roles": {
        "CONSULTANT": {"type": "LLM", "desc": "뉴스 텍스트 분석 및 토픽 스코어링"},
        "SUPERVISOR": {"type": "RULE", "desc": "AI 주식 관련성 임계치 평가"},
        "WORKER":     {"type": "ACTION", "desc": "최종 리포트 생성 또는 액션 수행"}
    },

    # 3. Transitions (전이 규칙 & 가드 조건)
    # snakes 코드의 net.add_transition() 및 net.add_input/output()에 해당
    # 코드의 if token... 로직은 'guard' 필드로 데이터화됨
    "transitions": {
        # Transition 1: Consultant
        # 코드: fire_if_enabled('consultant_transition', wrapped) -> 무조건 실행
        "consultant_transition": {
            "role": "CONSULTANT",
            "from_place": "incoming_news",
            "to_place":   "consulted",
            "guard": {
                "type": "ALWAYS", # 조건 없음
                "param": None
            }
        },

        # Transition 2: Supervisor (핵심 로직)
        # 코드: if token.data.get("AI_stocks", 0) > 0.8: ...
        "supervisor_transition": {
            "role": "SUPERVISOR",
            "from_place": "consulted",
            "to_place":   "evaluated",
            "guard": {
                "type": "THRESHOLD", # 임계치 체크 정책
                "param": {
                    "field": "AI_stocks",
                    "operator": 'gt',
                    "value": 0.8
                }
            }
        },

        # Transition 3: Worker
        # 코드: fire_if_enabled('worker_transition', token) -> evaluated에 있으면 실행
        "worker_transition": {
            "role": "WORKER",
            "from_place": "evaluated",
            "to_place":   "action_taken",
            "guard": {
                "type": "ALWAYS",
                "param": None
            }
        }
    }
}

### /ResourceSpecification

In [53]:
"""
목적 : 각 프로세스별로 넘어가는 Token 의 스키마 정의

시스템 데이터 무결성 보장을 위한 리소스 타입(Blueprints) 및 필수 필드 명세.
Raw Input(뉴스), Semantic Token(토픽 벡터), Directive(작업 지시)의 구조 표준화.
CPN Color Set 검증 기준이자 에이전트 간 인터페이스 계약(Contract) 역할 수행.
"""

RESOURCE_SPECS = {
    # 1. Input Layer (Consumed by Consultant)
    "RawNewsItem": {
        "spec_id": "RS_INPUT_001",
        "description": "비정형 원본 금융 뉴스 데이터",
        "schema": {
            "source_id": "str",       # 기사 고유 ID
            "content": "str",         # 본문 텍스트
            "publisher": "str",       # 출처 (Bloomberg, Reuters 등)
            "timestamp": "datetime"   # 발행 시각
        },
        "linked_topic_type": "General"
    },

    # 2. Semantic Layer (Produced by Consultant -> Consumed by Supervisor)
    "TopicWeightedToken": {
        "spec_id": "RS_TOKEN_001",
        "description": "LLM 분석을 통해 의미론적 가중치가 부여된 토큰 (CPN Core Token)",
        "schema": {
            "uuid": "str",            # 토큰 추적 ID (Traceability)
            "origin_ref": "str",      # 원본 소스 ID 참조
            "topic_vector": "dict",   # { "market_risk": 0.8, "tech_sector": 0.5 } - 희소 벡터
            "metadata": "json"        # LLM 모델명, 처리 시간, 신뢰도 점수 등
        },
        "linked_topic_type": "Financial_Semantic"
    },

    # 3. Execution Layer (Produced by Supervisor -> Consumed by Worker)
    "SupervisorDirective": {
        "spec_id": "RS_CMD_001",
        "description": "작업자(Worker)에게 전달되는 구체적 실행 명령",
        "schema": {
            "command_id": "str",      # 명령 ID
            "target_agent": "str",    # 수행 주체 (예: PortfolioManager)
            "action_type": "enum",    # [BUY, SELL, HOLD, ALERT]
            "parameters": "json",     # { "ticker": "NVDA", "amount": 100 }
            "reasoning": "str"        # 결정 근거 (Audit 용)
        },
        "linked_topic_type": "Decision_Control"
    }
}

### /functions

#### agents.py

In [5]:
"""AGENT_REGISTRY

AGENT 위계; 
CONSULTANT AGENT : 최초로 문제를 분석하여 구조화함
SUPERVISOR AGENT : 프로세스 흐름 제어 및 평가
WORKER AGENT     : 실제 액션 수행 (최종 결과물 생성)
"""

# 2. 에이전트 모의 실행 함수 (실제는 LangGraph/LangChain Invoke)
def agent_consultant(token):
    # TOPIC_DB의 지식을 활용한 분석 (Contextual Reasoning)
    sector_info = TOPIC_DB["taxonomy"].get("AI_sector", {}) 
    print(f"    [Consultant] {sector_info['description']} 기준 분석 시작...") 
    
    # 분석 결과를 토큰에 주입 (Coloring)
    token["domain_weight"] = sector_info.get("risk_weight", 1.0) 
    return token

def agent_supervisor(token):
    # 공유 DB의 임계치와 토큰의 스코어를 비교 (Formal Coordination)
    threshold = TOPIC_DB["taxonomy"]["AI_sector"]["critical_threshold"] 
    val = token.get('AI_stocks', 0)
    
    print(f"    [Supervisor] 공유 임계치({threshold}) 대비 현재값({val}) 평가") 
    
    # 평가 결과를 공유 DB의 동적 컨텍스트에 기록 (Shared State Update)
    if val > threshold:
        TOPIC_DB["dynamic_context"]["active_alerts"].append(f"High Risk: {token.get('text')[:10]}") 
        
    return token

def agent_worker(token):
    # 공유 DB의 최종 상태를 확인 후 액션 수행
    alerts = TOPIC_DB["dynamic_context"]["active_alerts"] 
    print(f"    [Worker] 공유 알림 목록 확인: {alerts}") 
    print("    [Worker] 최종 대응 보고서 및 액션 수행 완료")
    
    return {"status": "DONE", "processed_alerts": len(alerts)}


#### agents_registry.py

In [7]:
# 에이전트 레지스트리 (DB의 Role String -> Python Function 매핑)
AGENT_REGISTRY = {
    "CONSULTANT": agent_consultant,
    "SUPERVISOR": agent_supervisor,
    "WORKER":     agent_worker
}

#### functions.py

In [6]:
OPS = {
    "gt": operator.gt,  # >
    "lt": operator.lt,  # <
    "eq": operator.eq,  # ==
    "gte": operator.ge, # >=
    "lte": operator.le  # <=
}

# 3. 가드 평가 엔진 (핵심 로직)
def check_guard(guard_spec, token_data):
    g_type = guard_spec["type"]
    
    if g_type == "ALWAYS":
        return True
    
    elif g_type == "THRESHOLD":
        p = guard_spec["param"]
        field_val = token_data.get(p["field"], 0)
        target_val = p["value"]
        op_func = OPS.get(p["operator"])
        
        if not op_func:
            raise ValueError(f"Unknown operator: {p['operator']}")
            
        return op_func(field_val, target_val)
    
    return False

# 4. 메인 구동기 (The Engine)
def run_engine(process_spec, initial_data):
    # 초기 상태 설정
    current_place = process_spec["places"][0] # incoming_news
    token = initial_data
    steps = 0
    max_steps = 10 # 무한루프 방지
    
    print(f"프로세스 시작: {initial_data['text'][:20]}...")

    while steps < max_steps:
        print(f"\n Current Place: [{current_place}]")
        
        # 종료 조건: 더 이상 나갈 곳이 없는지 확인 (End Place)
        # 현재 위치에서 시작하는 Transition 검색
        possible_transitions = [
            t for t in process_spec["transitions"].values() if t["from_place"] == current_place
        ]
        
        if not possible_transitions:
            print(" 프로세스 종료 (No outgoing transitions).")
            break

        transition_triggered = False
        
        for t in possible_transitions:
            # 4-1. 가드 조건 평가 (결정론적 분기)
            if check_guard(t["guard"], token):
                print(f"    가드 통과: {t['role']} ({t['guard']['type']})")
                
                # 4-2. 에이전트 실행 (Role 위임)
                role = t["role"]
                agent_func = AGENT_REGISTRY.get(role)
                if agent_func:
                    # Token 업데이트
                    result = agent_func(token)
                    token.update(result)
                
                # 4-3. 토큰 이동 (상태 전이)
                current_place = t["to_place"]
                transition_triggered = True
                break # 결정론적이므로 하나 타면 루프 탈출
            else:
                print(f"   가드 실패: {t['role']}")

        if not transition_triggered:
            print(" Deadlock: 가능한 전이가 있으나 가드 조건 미달.")
            break
            
        steps += 1

    print(f" 최종 상태: {token}\n" + "="*30)

### main.py

In [10]:
# --- 실행 테스트 ---

"""
Token이란?

목적 : 사고 과정 로깅, 출처 표기, 상태 전달

토큰은 langraph의 State에 대응되는 것으로, 규격화된 데이터 구조를 통해 에이전트 프로세스 간에 전달됨
TASK_SPECIFICATION_DB는 프로세스 정의이고, token이 그 프로세스 내에서 이동하며 상태를 나타냄 (petrinet의 token과 유사)
비유하자면, TASK_SPECIFICATION_DB는 "철도"이고, token은 "기차"임. 
"기차"는 "철도" 위를 달리며, "철도"의 구조에 따라 움직임. 하지만 "기차" 자체는 독립적인 존재로, "철도"의 변화에 따라 움직임이 결정됨.
"기차" 내부는 여러 객차(데이터 필드)로 구성되어 있으며, 각 객차는 특정 정보를 담고 있음. 
역사학자가 앉아있는 객차도 존재하기 때문에, 우리는 기차를 읽으면 그것이 지나온 길도 파악할 수 있음. 
다만, "기차" 내부의 내용은 "철도"를 정의한다고 완전히 제어되는 것은 아님. 기차 내부는 여전히 확률론적인 답변을 따르기 때문임.
결국 확률론적 측면 제어는 현재 단순 로직으로만 구현돼있는 gurdrail의 고도화임. 향후 guardrail도 LLM 지능을 활용하는 방향으로 발전할 수 있음.

langraph state와 차이는 규격화된 데이터 구조를 사용한다는 점임 (딕셔너리 형태)
또한, state는 orchestrator 레벨의 LLM 판단 하에 관리되며, 각 에이전트 프로세스는 이 state를 받아 처리 후 반환하지만,
token의 경우 정해진 프로세스 내에서 직접 이동하고 변형됨. 따라서 에이전트 프로세스는 langraph와 다르게, 어느정도 "결정론적인" 프로세스를 따름.

현재는 token 내에 metadata 필드가 없지만, 향후 provenance 추적을 위해 추가할 예정임.
metadata 내에 사고 과정 및 데이터 출처 정보 기록시 token의 이동 경로를 추적할 수 있어, 투명성과 신뢰성을 높일 수 있음.
"""
token1 = {"text": "AI 호재 뉴스", "AI_stocks": 0.9}
token2 = {"text": "AI 악재 뉴스", "AI_stocks": 0.3}

# Case 1: 조건 만족 (0.9 > 0.8) -> 끝까지 실행
run_engine(TASK_SPECIFICATION_DB, token1)

# Case 2: 조건 불만족 (0.3 < 0.8) -> Deadlock (Supervisor에서 멈춤)
run_engine(TASK_SPECIFICATION_DB, token2)

프로세스 시작: AI 호재 뉴스...

 Current Place: [incoming_news]
    가드 통과: CONSULTANT (ALWAYS)
    [Consultant] Artificial Intelligence and Semiconductor market 기준 분석 시작...

 Current Place: [consulted]
    가드 통과: SUPERVISOR (THRESHOLD)
    [Supervisor] 공유 임계치(0.85) 대비 현재값(0.9) 평가

 Current Place: [evaluated]
    가드 통과: WORKER (ALWAYS)
    [Worker] 공유 알림 목록 확인: ['High Risk: AI 호재 뉴스']
    [Worker] 최종 대응 보고서 및 액션 수행 완료

 Current Place: [action_taken]
 프로세스 종료 (No outgoing transitions).
 최종 상태: {'text': 'AI 호재 뉴스', 'AI_stocks': 0.9, 'domain_weight': 1.5, 'status': 'DONE', 'processed_alerts': 1}
프로세스 시작: AI 악재 뉴스...

 Current Place: [incoming_news]
    가드 통과: CONSULTANT (ALWAYS)
    [Consultant] Artificial Intelligence and Semiconductor market 기준 분석 시작...

 Current Place: [consulted]
   가드 실패: SUPERVISOR
 Deadlock: 가능한 전이가 있으나 가드 조건 미달.
 최종 상태: {'text': 'AI 악재 뉴스', 'AI_stocks': 0.3, 'domain_weight': 1.5}


### 손타자로 로직 이해

In [ ]:
def agent_consultant(token):
    sector_info = TOPIC_DB['taxonomy']
    token["domain_weight"] = sector_info.get("risk_weight", 1.0)
    return token

def agent_supervisor(token):
    threshold = TOPIC_DB['taxonomy']['AI_sector']['critical_threshold']
    val = token.get('AI_stocks', 0)

    if val > threshold:
        TOPIC_DB['dynamic_context']['active_alerts'].append(f"High Risk:{token.get('text')[:10]}")

def agent_worker(token):
    alerts = TOPIC_DB['dynamic_context']['active_alerts']
    print(f"[Worker] 공유 알림 목록 확인: {alerts}")
    print(f"[Worker] 최종 대응 보고서 및 액션 수행 완료")

    return {"status":"DONE", "processed_alerts" : len(alerts)}

In [ ]:
def check_guard(guard_spec, token_data):
    g_type = guard_spec['type']
    
    if g_type == "ALWAYS":
        return True
    elif g_type == "THRESHOLD":
        p = guard_spec['param']
        field_val = token_data.get(p['field'], 0)
        target_val = p['value']
        op_func = OPS.get(p['operator'])

        if not op_func:
            raise ValueError(f"Unknown operator : {p['operator']}")
        
        return op_func(field_val, target_val)
    else:    
        return False

In [16]:
def run_engine(process_spec, initial_data):
    # 초기 상태 설정
    current_place = process_spec["places"][0]
    token = initial_data
    steps = 0
    max_steps = 10 # 무한 루프 방지

    print(f"프로세스 시작")

    while steps < max_steps:
        # 현재 "PLACE" 기술
        print(f"\n Current Place: [{current_place}]")
        
        # 현재 위치에서 시작하는 Transition 검색
        possible_transitions = [
            t for t in process_spec['transitions'].values() if t['from_place'] == current_place
        ]

        if not possible_transitions:
            print("프로세스 종료 (No Outgoing Transitions)")
            break

        # gurad pass 해야 상태 전이 허용
        # 현재 허용 상태를 사전 정의
        transition_triggered = False

        for t in possible_transitions:
            # 4-1. 가드 조건 평가 (결정론적 분기)
            if check_guard(t["guard"], token): # TRUE / FALSE 리턴. 조건 분기를 함수로!
                print(f"    가드 통과: {t['role']} ({t['guard']['type']})")

                # 4-2. 에이전트 Role 지정 및 토큰 통과
                role = t['role']
                agent_func = AGENT_REGISTRY.get(role)
                if agent_func:
                    result = agent_func(token)
                    token.update(result) # 동일한 key가 있으면 value update, 없으면 통째로 업데이트
                
                # 4-3. 토큰 이동
                current_place = t["to_place"]
                transition_triggered = True
                break
            else:
                print(f" 가드 실패 : {t['role']}")

        if not transition_triggered:
            print("DeadLock")
            break
        step += 1
    print(f"최종 상태: {token}")




In [15]:
TASK_SPECIFICATION_DB['transitions'].values()

dict_values([{'role': 'CONSULTANT', 'from_place': 'incoming_news', 'to_place': 'consulted', 'guard': {'type': 'ALWAYS', 'param': None}}, {'role': 'SUPERVISOR', 'from_place': 'consulted', 'to_place': 'evaluated', 'guard': {'type': 'THRESHOLD', 'param': {'field': 'AI_stocks', 'operator': 'gt', 'value': 0.8}}}, {'role': 'WORKER', 'from_place': 'evaluated', 'to_place': 'action_taken', 'guard': {'type': 'ALWAYS', 'param': None}}])